In [10]:
# Instal PyTorch dan library pendukung lainnya
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
%pip install timm pandas numpy matplotlib seaborn scikit-learn tqdm pillow

Looking in indexes: https://download.pytorch.org/whl/cu118Note: you may need to restart the kernel to use updated packages.




In [11]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, precision_score, recall_score, f1_score

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models

from PIL import Image
import timm  # Untuk mengakses model ViT

import random
from tqdm import tqdm
import time  # Untuk debugging timeout

# Set random seed untuk reproducibility
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

set_seed()

# Cek ketersediaan GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Menggunakan perangkat: {device}")
print(f"PyTorch version: {torch.__version__}")
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"GPU device: {torch.cuda.get_device_name(0)}")

# Konfigurasi umum
BATCH_SIZE = 16  # Mengurangi batch size untuk menghindari OOM error
IMAGE_SIZE = 224  # ViT-B/16 input size
NUM_EPOCHS = 20
LEARNING_RATE = 1e-4
CLASSES = ["Chicken_Coccidiosis", "Chicken_Healthy", "Chicken_NewCastleDisease", "Chicken_Salmonella"]
NUM_CLASSES = len(CLASSES)


Menggunakan perangkat: cpu
PyTorch version: 2.6.0+cu118


In [12]:
# 1. DATASET PREPARATION
class ChickenFecesDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        """
        Dataset kustom untuk dataset kotoran ayam
        Args:
            data_dir (str): Path ke direktori dataset
            transform: Transformasi yang akan diterapkan pada gambar
        """
        self.data_dir = data_dir
        self.transform = transform
        self.classes = CLASSES
        self.class_to_idx = {cls: idx for idx, cls in enumerate(self.classes)}
        
        self.samples = []
        
        # Mencari semua file gambar dalam direktori data
        for class_name in self.classes:
            class_dir = os.path.join(data_dir, class_name)
            if os.path.isdir(class_dir):
                class_files = [f for f in os.listdir(class_dir) 
                            if os.path.isfile(os.path.join(class_dir, f)) and 
                            f.lower().endswith(('.png', '.jpg', '.jpeg'))]
                print(f"Kelas {class_name}: {len(class_files)} gambar ditemukan")
                
                for img_name in class_files:
                    img_path = os.path.join(class_dir, img_name)
                    self.samples.append((img_path, self.class_to_idx[class_name]))
            else:
                print(f"PERINGATAN: Direktori {class_dir} tidak ditemukan!")
        
        if not self.samples:
            raise RuntimeError(f"Tidak ada gambar yang ditemukan di {data_dir}. Pastikan struktur folder benar.")
        
        print(f"Total dataset: {len(self.samples)} gambar")
    
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        
        try:
            # Buka gambar dan konversi ke RGB (untuk memastikan 3 channel)
            image = Image.open(img_path).convert('RGB')
            
            # Terapkan transformasi jika ada
            if self.transform:
                image = self.transform(image)
                
            return image, label
        except Exception as e:
            print(f"Error loading image {img_path}: {e}")
            # Return placeholder image jika terjadi error
            dummy_img = torch.zeros((3, IMAGE_SIZE, IMAGE_SIZE))
            return dummy_img, label

In [13]:
# 2. DATA PREPROCESSING & AUGMENTATION
# Transformasi untuk training (dengan augmentasi)
train_transform = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.2),
    transforms.RandomRotation(degrees=15),
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.05),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # ImageNet stats
])

# Transformasi untuk validasi/testing (tanpa augmentasi)
val_transform = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # ImageNet stats
])

In [14]:
# 3. MEMBUAT DATA LOADERS
def create_dataloaders(data_dir, train_ratio=0.8):
    """
    Membuat dan mengembalikan DataLoader untuk training dan validasi
    Args:
        data_dir: Path ke direktori dataset
        train_ratio: Rasio untuk data training vs. validasi
    """
    print(f"Membuat dataset dari direktori: {data_dir}")
    
    # Verifikasi direktori dataset
    if not os.path.exists(data_dir):
        raise FileNotFoundError(f"Direktori dataset tidak ditemukan: {data_dir}")
    
    # Lihat struktur direktori
    print("Struktur direktori dataset:")
    for item in os.listdir(data_dir):
        item_path = os.path.join(data_dir, item)
        if os.path.isdir(item_path):
            print(f"  - {item}/")
            files = os.listdir(item_path)[:5]  # Ambil 5 file pertama sebagai contoh
            for f in files:
                print(f"    - {f}")
            if len(os.listdir(item_path)) > 5:
                print(f"    - ... dan {len(os.listdir(item_path))-5} file lainnya")
    
    # Membuat dataset penuh
    try:
        full_dataset = ChickenFecesDataset(data_dir, transform=None)
    except Exception as e:
        print(f"Error saat membuat dataset: {e}")
        raise
    
    # Menghitung ukuran split
    dataset_size = len(full_dataset)
    train_size = int(train_ratio * dataset_size)
    val_size = dataset_size - train_size
    
    print(f"Membagi dataset: {train_size} untuk training, {val_size} untuk validasi")
    
    # Membagi dataset
    indices = list(range(dataset_size))
    random.shuffle(indices)
    train_indices = indices[:train_size]
    val_indices = indices[train_size:train_size + val_size]
    
    # Subset sampel berdasarkan indeks
    train_samples = [full_dataset.samples[i] for i in train_indices]
    val_samples = [full_dataset.samples[i] for i in val_indices]
    
    # Membuat dataset untuk training dan validasi
    train_dataset = ChickenFecesDataset(data_dir, transform=train_transform)
    train_dataset.samples = train_samples
    
    val_dataset = ChickenFecesDataset(data_dir, transform=val_transform)
    val_dataset.samples = val_samples
    
    print("Membuat DataLoader...")
    
    # Membuat DataLoader dengan pengaturan aman untuk Colab
    # Kurangi num_workers dan hapus pin_memory untuk mencegah hanging
    train_loader = DataLoader(
        train_dataset, 
        batch_size=BATCH_SIZE, 
        shuffle=True, 
        num_workers=0,  # Gunakan 0 untuk menghindari masalah
        pin_memory=False  # Matikan pin_memory
    )
    
    val_loader = DataLoader(
        val_dataset, 
        batch_size=BATCH_SIZE, 
        shuffle=False, 
        num_workers=0,  # Gunakan 0 untuk menghindari masalah
        pin_memory=False  # Matikan pin_memory
    )
    
    # Verifikasi DataLoader
    print(f"Train loader: {len(train_loader)} batches")
    print(f"Val loader: {len(val_loader)} batches")
    
    # Tes DataLoader dengan load batch pertama 
    print("Tes DataLoader (mencoba load batch pertama)...")
    try:
        start_time = time.time()
        train_iter = iter(train_loader)
        first_batch = next(train_iter)
        print(f"Batch pertama berhasil di-load dalam {time.time() - start_time:.2f} detik")
        print(f"Batch shape: {first_batch[0].shape}, Labels: {first_batch[1]}")
    except Exception as e:
        print(f"Error saat mencoba load batch pertama: {e}")
    
    return train_loader, val_loader

In [15]:
# 4. MODEL ARCHITECTURE - VISION TRANSFORMER (ViT-B/16)
class ViTChickenClassifier(nn.Module):
    def __init__(self, num_classes=NUM_CLASSES):
        super(ViTChickenClassifier, self).__init__()
        
        # Memuat ViT-B/16 pretrained
        print("Memuat model ViT-B/16 pretrained...")
        try:
            self.vit = timm.create_model('vit_base_patch16_224', pretrained=True)
            print("Model ViT berhasil dimuat")
        except Exception as e:
            print(f"Error saat memuat model ViT: {e}")
            raise
        
        # Mengganti head untuk klasifikasi kustom
        in_features = self.vit.head.in_features
        self.vit.head = nn.Linear(in_features, num_classes)
        print(f"Head classifier dimodifikasi: {in_features} -> {num_classes}")
        
    def forward(self, x):
        return self.vit(x)

In [16]:
# 5. TRAINING FUNCTION
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler=None, num_epochs=NUM_EPOCHS):
    """Melatih model dan melakukan validasi"""
    best_val_accuracy = 0.0
    history = {'train_loss': [], 'val_loss': [], 'train_acc': [], 'val_acc': []}
    
    for epoch in range(num_epochs):
        print(f"\nEpoch {epoch+1}/{num_epochs} dimulai")
        epoch_start_time = time.time()
        
        # === TRAINING PHASE ===
        model.train()
        running_loss = 0.0
        train_correct = 0
        train_total = 0
        
        train_loop = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs} [Train]')
        batch_count = 0
        
        for inputs, labels in train_loop:
            batch_start = time.time()
            inputs, labels = inputs.to(device), labels.to(device)
            
            # Forward pass
            optimizer.zero_grad()
            
            try:
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                
                # Backward pass
                loss.backward()
                optimizer.step()
                
                # Statistik
                running_loss += loss.item() * inputs.size(0)
                _, predicted = torch.max(outputs.data, 1)
                train_total += labels.size(0)
                train_correct += (predicted == labels).sum().item()
                
                # Update tqdm description dengan loss saat ini
                batch_count += 1
                if batch_count % 10 == 0:  # Log setiap 10 batch untuk mengurangi output
                    train_loop.set_postfix(
                        loss=loss.item(), 
                        accuracy=(train_correct/train_total)*100,
                        time_per_batch=f"{(time.time()-batch_start):.2f}s"
                    )
                
            except Exception as e:
                print(f"Error pada batch training: {e}")
                continue
        
        # Menghitung metrik training
        epoch_train_loss = running_loss / len(train_loader.dataset)
        epoch_train_acc = 100.0 * train_correct / train_total
        
        # Update scheduler jika disediakan
        if scheduler:
            scheduler.step()
        
        # === VALIDATION PHASE ===
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        
        all_preds = []
        all_labels = []
        
        with torch.no_grad():
            val_loop = tqdm(val_loader, desc=f'Epoch {epoch+1}/{num_epochs} [Val]')
            for inputs, labels in val_loop:
                try:
                    inputs, labels = inputs.to(device), labels.to(device)
                    
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    
                    val_loss += loss.item() * inputs.size(0)
                    _, predicted = torch.max(outputs.data, 1)
                    val_total += labels.size(0)
                    val_correct += (predicted == labels).sum().item()
                    
                    # Menyimpan prediksi dan label untuk confusion matrix
                    all_preds.extend(predicted.cpu().numpy())
                    all_labels.extend(labels.cpu().numpy())
                    
                    # Update tqdm description
                    val_loop.set_postfix(loss=loss.item(), accuracy=(val_correct/val_total)*100)
                except Exception as e:
                    print(f"Error pada batch validasi: {e}")
                    continue
        
        # Menghitung metrik validasi
        epoch_val_loss = val_loss / len(val_loader.dataset)
        epoch_val_acc = 100.0 * val_correct / val_total
        
        # Menyimpan metrik
        history['train_loss'].append(epoch_train_loss)
        history['val_loss'].append(epoch_val_loss)
        history['train_acc'].append(epoch_train_acc)
        history['val_acc'].append(epoch_val_acc)
        
        epoch_time = time.time() - epoch_start_time
        print(f'Epoch {epoch+1}/{num_epochs} selesai dalam {epoch_time:.2f} detik:')
        print(f'Train Loss: {epoch_train_loss:.4f}, Train Accuracy: {epoch_train_acc:.2f}%')
        print(f'Val Loss: {epoch_val_loss:.4f}, Val Accuracy: {epoch_val_acc:.2f}%')
        
        # Menyimpan model terbaik berdasarkan akurasi validasi
        if epoch_val_acc > best_val_accuracy:
            best_val_accuracy = epoch_val_acc
            try:
                torch.save(model.state_dict(), 'best_vit_chicken_classifier.pth')
                print(f'Model disimpan dengan akurasi validasi: {best_val_accuracy:.2f}%')
            except Exception as e:
                print(f"Error saat menyimpan model: {e}")
            
            # Evaluasi model terbaik
            if epoch == num_epochs - 1 or epoch_val_acc > 95:  # Evaluasi di akhir atau jika akurasi sangat tinggi
                print("\nModel Evaluation:")
                try:
                    cm = confusion_matrix(all_labels, all_preds)
                    
                    # Menghitung metrik per kelas
                    precision = precision_score(all_labels, all_preds, average=None)
                    recall = recall_score(all_labels, all_preds, average=None)
                    f1 = f1_score(all_labels, all_preds, average=None)
                    
                    print("\nConfusion Matrix:")
                    print(cm)
                    
                    print("\nClassification Report:")
                    print(classification_report(all_labels, all_preds, target_names=CLASSES))
                    
                    # Plot confusion matrix
                    plt.figure(figsize=(10, 8))
                    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=CLASSES, yticklabels=CLASSES)
                    plt.xlabel('Predicted')
                    plt.ylabel('True')
                    plt.title('Confusion Matrix')
                    plt.savefig('confusion_matrix.png')
                    plt.close()
                    
                    # Plot training history
                    plt.figure(figsize=(12, 5))
                    
                    plt.subplot(1, 2, 1)
                    plt.plot(history['train_loss'], label='Train Loss')
                    plt.plot(history['val_loss'], label='Validation Loss')
                    plt.title('Loss')
                    plt.xlabel('Epoch')
                    plt.ylabel('Loss')
                    plt.legend()
                    
                    plt.subplot(1, 2, 2)
                    plt.plot(history['train_acc'], label='Train Accuracy')
                    plt.plot(history['val_acc'], label='Validation Accuracy')
                    plt.title('Accuracy')
                    plt.xlabel('Epoch')
                    plt.ylabel('Accuracy (%)')
                    plt.legend()
                    
                    plt.tight_layout()
                    plt.savefig('training_history.png')
                    plt.close()
                except Exception as e:
                    print(f"Error saat evaluasi model: {e}")
    
    return history

In [17]:
# 6. FINE-TUNING STRATEGY
def setup_finetuning(model, freeze_backbone=True):
    """Mengatur strategi fine-tuning untuk model"""
    
    if freeze_backbone:
        # Membekukan (freeze) semua parameter dalam backbone
        print("Membekukan backbone...")
        for name, param in model.named_parameters():
            if 'head' not in name:  # Membekukan semua layer kecuali head
                param.requires_grad = False
    
    # Memeriksa parameter yang dapat dilatih
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    total_params = sum(p.numel() for p in model.parameters())
    
    print(f'Total parameters: {total_params:,}')
    print(f'Trainable parameters: {trainable_params:,} ({trainable_params/total_params:.2%})')
    
    return model


In [18]:
# 7. MAIN FUNCTION
def main(data_dir):
    """Fungsi utama untuk melatih dan mengevaluasi model"""
    
    print("\n=== MULAI TRAINING ===")
    
    # Membuat data loaders
    try:
        train_loader, val_loader = create_dataloaders(data_dir)
    except Exception as e:
        print(f"Error fatal saat membuat DataLoader: {e}")
        return None, None
    
    # Membuat model
    try:
        model = ViTChickenClassifier(num_classes=NUM_CLASSES)
        model = setup_finetuning(model, freeze_backbone=True)  # Freeze backbone awalnya
        model = model.to(device)
    except Exception as e:
        print(f"Error fatal saat mempersiapkan model: {e}")
        return None, None
    
    # Mendefinisikan loss function dan optimizer
    criterion = nn.CrossEntropyLoss()
    
    # FASE 1: Melatih hanya head (classifier) selama beberapa epoch
    optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=LEARNING_RATE)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)
    
    print("=== FASE 1: Fine-tuning head classifier ===")
    history = train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs=5)
    
    # FASE 2: Unfreeze beberapa layer terakhir dan lanjutkan training
    # Memunculkan (unfreeze) beberapa layer terakhir untuk fine-tuning lebih lanjut
    print("\nUnfreezing beberapa layer terakhir...")
    for name, param in model.named_parameters():
        if 'blocks.11' in name or 'blocks.10' in name or 'head' in name:
            param.requires_grad = True
    
    # Memeriksa parameter yang dapat dilatih setelah unfreezing
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    total_params = sum(p.numel() for p in model.parameters())
    print(f'Trainable parameters setelah unfreezing: {trainable_params:,} ({trainable_params/total_params:.2%})')
    
    # Optimizer baru dengan learning rate yang lebih kecil untuk fine-tuning
    optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=LEARNING_RATE/10)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=15)
    
    print("\n=== FASE 2: Fine-tuning layer terakhir ===")
    history = train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs=NUM_EPOCHS-5)
    
    print("Training selesai!")
    return model, history

# Menjalankan program jika dieksekusi langsung
if __name__ == "__main__":
    # Isi dengan path dataset Anda
    DATA_DIR = "chicken_feces_dataset"  # Path default di Colab setelah extract
    
    try:
        model, history = main(DATA_DIR)
    except Exception as e:
        print(f"Error selama eksekusi: {e}")


=== MULAI TRAINING ===
Membuat dataset dari direktori: chicken_feces_dataset
Struktur direktori dataset:
  - Chicken_Coccidiosis/
    - cocci.0.jpg
    - cocci.1.jpg
    - cocci.10.jpg
    - cocci.100.jpg
    - cocci.1000.jpg
    - ... dan 2397 file lainnya
  - Chicken_Healthy/
    - healthy.0.jpg
    - healthy.1.jpg
    - healthy.10.jpg
    - healthy.100.jpg
    - healthy.1000.jpg
    - ... dan 2326 file lainnya
  - Chicken_NewCastleDisease/
    - ncd.0.jpg
    - ncd.1.jpg
    - ncd.10.jpg
    - ncd.100.jpg
    - ncd.101.jpg
    - ... dan 520 file lainnya
  - Chicken_Salmonella/
    - pcrsalmo.0.jpg
    - pcrsalmo.1.jpg
    - pcrsalmo.10.jpg
    - pcrsalmo.100.jpg
    - pcrsalmo.101.jpg
    - ... dan 2164 file lainnya
Kelas Chicken_Coccidiosis: 2402 gambar ditemukan
Kelas Chicken_Healthy: 2331 gambar ditemukan
Kelas Chicken_NewCastleDisease: 525 gambar ditemukan
Kelas Chicken_Salmonella: 2169 gambar ditemukan
Total dataset: 7427 gambar
Membagi dataset: 5941 untuk training, 1486 untuk

Epoch 1/5 [Val]: 100%|██████████| 93/93 [05:07<00:00,  3.30s/it, accuracy=86.8, loss=0.523]


Epoch 1/5 selesai dalam 1449.21 detik:
Train Loss: 0.6373, Train Accuracy: 77.29%
Val Loss: 0.4011, Val Accuracy: 86.81%
Model disimpan dengan akurasi validasi: 86.81%

Epoch 2/5 dimulai


Epoch 2/5 [Val]: 100%|██████████| 93/93 [03:58<00:00,  2.57s/it, accuracy=91.6, loss=0.426]


Epoch 2/5 selesai dalam 1331.21 detik:
Train Loss: 0.3177, Train Accuracy: 90.14%
Val Loss: 0.2796, Val Accuracy: 91.59%
Model disimpan dengan akurasi validasi: 91.59%

Epoch 3/5 dimulai


...

Epoch 13/15 selesai dalam 1452.24 detik:
Train Loss: 0.0178, Train Accuracy: 99.41%
Val Loss: 0.0890, Val Accuracy: 97.85%
Model disimpan dengan akurasi validasi: 97.85%

Model Evaluation:

Confusion Matrix:
[[482   5   1   1]
 [  6 447   1   7]
 [  1   2 102   0]
 [  0   7   1 423]]

Classification Report:
                          precision    recall  f1-score   support

     Chicken_Coccidiosis       0.99      0.99      0.99       489
         Chicken_Healthy       0.97      0.97      0.97       461
Chicken_NewCastleDisease       0.97      0.97      0.97       105
      Chicken_Salmonella       0.98      0.98      0.98       431

                accuracy                           0.98      1486
               macro avg       0.98      0.98      0.98      1486
            weighted avg       0.98      0.98      0.98      1486


Epoch 14/15 dimulai


Epoch 14/15 [Val]: 100%|██████████| 93/93 [03:35<00:00,  2.31s/it, accuracy=97.5, loss=0.0677]  


Epoch 14/15 selesai dalam 1499.18 detik:
Train Loss: 0.0161, Train Accuracy: 99.53%
Val Loss: 0.0899, Val Accuracy: 97.51%

Epoch 15/15 dimulai


Epoch 15/15 [Val]: 100%|██████████| 93/93 [03:40<00:00,  2.37s/it, accuracy=97.4, loss=0.07]    

Epoch 15/15 selesai dalam 1518.12 detik:
Train Loss: 0.0140, Train Accuracy: 99.66%
Val Loss: 0.0901, Val Accuracy: 97.44%
Training selesai!
